## How can I leverage State-of-the-Art Natural Language Models with only one line of code?
Pipelines encapsulate the overall process of every NLP process:

Tokenization: Split the initial input into multiple sub-entities with ... properties (i.e. tokens).
Inference: Maps every tokens into a more meaningful representation.
Decoding: Use the above representation to generate and/or extract the final output for the underlying task.

```python
from transformers import pipeline

# Using default model and tokenizer for the task
pipeline("<task-name>")

# Using a user-specified model
pipeline("<task-name>", model="<model_name>")

# Using custom model/tokenizer as str
pipeline('<task-name>', model='<model name>', tokenizer='<tokenizer_name>')
```

In [0]:
!pip install -q transformers

In [0]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline

## Sentence Classification - Sentiment Analysis

In [7]:
nlp_sentence_classif = pipeline('sentiment-analysis')
nlp_sentence_classif('Corona is killing people')

[{'label': 'NEGATIVE', 'score': 0.9983923}]

## Token Classification - Named Entity Recognition

In [8]:
nlp_token_class = pipeline('ner')
nlp_token_class('I love Ariana Grande. She is a great singer')

[{'entity': 'I-PER', 'score': 0.9982187747955322, 'word': 'Aria'},
 {'entity': 'I-PER', 'score': 0.998218297958374, 'word': '##na'},
 {'entity': 'I-PER', 'score': 0.9991388320922852, 'word': 'Grande'}]

## Question Answering

In [11]:
nlp_qa = pipeline('question-answering')
nlp_qa(context='I love Ariana Grande. She is a great singer', question='Why do I like her?')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 630.06it/s]


{'answer': 'She is a great singer',
 'end': 42,
 'score': 0.3110909627069134,
 'start': 22}

## Text Generation - Mask Filling

In [15]:
nlp_fill = pipeline('fill-mask')
nlp_fill('Now give people back their' + nlp_fill.tokenizer.mask_token)

[{'score': 0.07639964669942856,
  'sequence': '<s> Now give people back their homes</s>',
  'token': 1611},
 {'score': 0.05069264769554138,
  'sequence': '<s> Now give people back their money</s>',
  'token': 418},
 {'score': 0.045410484075546265,
  'sequence': '<s> Now give people back their lives</s>',
  'token': 1074},
 {'score': 0.018801607191562653,
  'sequence': '<s> Now give people back their wallets</s>',
  'token': 28811},
 {'score': 0.01855502463877201,
  'sequence': '<s> Now give people back their memories</s>',
  'token': 6180}]

## Summarization

Summarization is currently supported by `Bart` and `T5`.

In [18]:
# Favourite Part of Me - Astrid S
TEXT_TO_SUMMARIZE = """
Having a bad day, having a good day
Having a day that's in between
Feel like I hold back and I don't know why
Can't seem to say just what I mean
So many people, all of the ones surrounding me
And if you'd ask them, they would describe me differently
So many reasons, so many different sides of me
And I can't decide who I want to be
'Cause some people make me bitter
Smile and then squeeze a lime in my cut
And some people make me sweeter
'Cause they make me feel like I am enough
And that makes me a giver
But my mom says I tend to give too much
And some people make me better
And that's how I know my favorite part of me is you
My favorite part of me is you
Having a moment, weight on my shoulders
I can't describe just how I feel
Don't have to say it, look at my face and
You know exactly what I mean
So many people, all of the ones surrounding me
And if you'd ask them, they would describe me differently
So many reasons, so many different sides of me
And I can't decide who I want to be
'Cause some people make me bitter
Smile and then squeeze a lime in my cut
And some people make me sweeter
'Cause they make me feel like I am enough
And that makes me a giver
But my mom says I tend to give too much
And some people make me better
And that's how I know my favorite part of me is you
My favorite part of me is you
My favorite part of me is you
My favorite part of me is you
Some people make me nervous
'Cause they make me feel like nothing at all
But some people think I'm perfect
'Cause they wouldn't want me to change at all
And that's what I deserve
And I'm reminded of how lucky I am
'Cause most people make me better
And that's how I know
My favorite part of me is you
My favorite part of me is you
My favorite part of me is you
My favorite part of me is you
"""

summarizer = pipeline('summarization')
summarizer(TEXT_TO_SUMMARIZE)

Couldn't reach server at 'https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/modelcard.json' to download model card file.
Creating an empty model card.


[{'summary_text': "So many people, all of the ones surrounding me, would describe me differently. Some people make me bitter and then squeeze a lime in my cut. Others make me sweeter and that makes me a giver. My mom says I tend to give too much. And that's how I know my favorite part of me is you."}]

## 6. Translation

Translation is currently supported by `T5` for the language mappings English-to-French (`translation_en_to_fr`), English-to-German (`translation_en_to_de`) and English-to-Romanian (`translation_en_to_ro`).

In [21]:
# English to French
translator = pipeline('translation_en_to_fr')
translator("So many people, all of the ones surrounding me, would describe me differently")

[{'translation_text': "De nombreuses personnes, toutes celles qui m'entourent, me décriraient différemment."}]

## Projection - Features Extraction 

In [22]:
import numpy as np
nlp_features = pipeline('feature-extraction')
output = nlp_features('Hugging Face is a French company based in Paris')
np.array(output).shape   # (Samples, Tokens, Vector Size)

(1, 12, 768)

Alright ! Now you have a nice picture of what is possible through transformers' pipelines, and there is more
to come in future releases. 

In the meantime, you can try the different pipelines with your own inputs

In [24]:
task = widgets.Dropdown(
    options=['sentiment-analysis', 'ner', 'fill_mask'],
    value='ner',
    description='Task:',
    disabled=False
)

input = widgets.Text(
    value='',
    placeholder='Enter something',
    description='Your input:',
    disabled=False
)

def forward(_):
    if len(input.value) > 0: 
        if task.value == 'ner':
            output = nlp_token_class(input.value)
        elif task.value == 'sentiment-analysis':
            output = nlp_sentence_classif(input.value)
        else:
            if input.value.find('<mask>') == -1:
                output = nlp_fill(input.value + ' <mask>')
            else:
                output = nlp_fill(input.value)                
        print(output)

input.on_submit(forward)
display(task, input)

Dropdown(description='Task:', index=1, options=('sentiment-analysis', 'ner', 'fill_mask'), value='ner')

Text(value='', description='Your input:', placeholder='Enter something')

[{'sequence': '<s> had a couple beers last nig</s>', 'score': 0.21246689558029175, 'token': 16328}, {'sequence': '<s> had a couple drinks last nig</s>', 'score': 0.10781652480363846, 'token': 6696}, {'sequence': '<s> had a couple nights last nig</s>', 'score': 0.016789980232715607, 'token': 7011}, {'sequence': '<s> had a couple times last nig</s>', 'score': 0.014156781136989594, 'token': 498}, {'sequence': '<s> had a couple days last nig</s>', 'score': 0.013142986223101616, 'token': 360}]


In [25]:
context = widgets.Textarea(
    value='Einstein is famous for the general theory of relativity',
    placeholder='Enter something',
    description='Context:',
    disabled=False
)

query = widgets.Text(
    value='Why is Einstein famous for ?',
    placeholder='Enter something',
    description='Question:',
    disabled=False
)

def forward(_):
    if len(context.value) > 0 and len(query.value) > 0: 
        output = nlp_qa(question=query.value, context=context.value)            
        print(output)

query.on_submit(forward)
display(context, query)

Textarea(value='Einstein is famous for the general theory of relativity', description='Context:', placeholder=…

Text(value='Why is Einstein famous for ?', description='Question:', placeholder='Enter something')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4940.29it/s]


{'score': 0.40340638286807007, 'start': 27, 'end': 54, 'answer': 'general theory of relativity'}
